In [1]:
# Dependencies

import numpy as np
import pandas as pd
import random 
import matplotlib.pyplot as plt

In [2]:
'''
Data picked: https://www.kaggle.com/c/house-prices-advanced-regression-techniques
'''
df = pd.read_csv("/Users/neha.arora/Downloads/train.csv", header=None)
dfTrain = df.filter([3,80], axis=1).loc[1:].dropna()
dfTrain = dfTrain.values.astype(float)

df2 = pd.read_csv("/Users/neha.arora/Downloads/train.csv", header=None)
dfTest = df2.filter([3], axis=1).loc[1:].dropna()
dfTest = dfTest.values.astype(float)

weights = np.zeros(2)

In [3]:
'''
The independent variable takes into account the feature:
GrLivArea: Above grade (ground) living area square feet

Variable being predicted: SalePrice

Plots to analyze the relationship of each variable with the variable being predicted
'''

import matplotlib.pyplot as plt

plt.figure(1)

plt.scatter(dfTrain[:,0], dfTrain[:,1])
plt.title("Living Area vs Cost of House")

plt.show()

In [4]:
'''
Feature scaling (normalization) to ensure proportionate contribution of features to objective function
X(i) = (X(i) - mean(X(i)))/std(X(i)) , X(i) represents the i th feature of dataset
'''

def normalizeFeatures(X):
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    normalizedX = (X-mean)/std
    return normalizedX


In [5]:
'''
Predict the value(s) for given data point(s) X and weight vector theta
h(theta, X) = X * theta'
where h(theta, X) is the value based on hypothesis given theta and X

X : [m x (n+1)]
theta : [1 x (n+1)]
(m: examples in data set
 n: feature space of X)
 
Returns predicted values y
y : [m x 1]
'''
def predict(X, theta):
    y = np.dot(X, theta.T)
    return y

In [6]:
'''
Objective function that needs to be minimized for optimal solution
Error calculation using SSE: Sum of Squared Errors
E = 1/(2*m) * ((h(theta,X) - y) ** 2)

Returns the sum of squared errors value for the data point(s) X
'''

def SSE(X, theta, y):
    predicted = predict(X, theta)
    error = predicted - y
    squaredErrors = np.dot(error.T, error)
    sse = np.sum(squaredErrors)/(2*X.shape[0])
    return sse

In [7]:
'''
Gradient calculation for the objective function
(E : SSE, 
theta : w (the weight vector))

dE/dw = dE/dh * dh/dw
(h : predict function for given X and theta (w))

dE/dw(i) = 1/(2*m) * (-2 * (h(theta, X) - y)) * dh/dw(i)
dE/dw(i) = -1/m * (h(theta, X) - y)) * X(i)

Returns a vector of gradients

'''
def gradient(X, theta, y):
    grad = np.dot(X.T, predict(X, theta) - y)/X.shape[0]
    return grad.T

In [8]:
'''
Gradient Descent to minimize loss

Two approaches:
1) Batch Gradient Descent: Use all the examples of the training set to determine the update in weight parameters
2) Stochastic Gradient Descent: Use one example of the training set at a time to update weight parameters
'''
def gradient_descent_step(X, theta, y, alpha):
    grad = gradient(X, theta, y)
    theta = theta - alpha * grad
    return theta

def batch_gradient_descent (X, theta, y, num_iters, alpha):
    error_record = np.zeros(num_iters)
    for iter in range(num_iters):
        error_record[iter] = SSE(X, theta, y)
        theta = gradient_descent_step(X, theta, y, alpha)
    return [theta, error_record]

def stochastic_gradient_descent (X, theta, y, num_iters, alpha):
    error_record = np.zeros(num_iters * X.shape[0])
    counter = 0
    for rec in range(X.shape[0]):
        for iter in range(num_iters):
            error_record[counter] = SSE(X, theta, y)
            counter = counter+1
            theta = gradient_descent_step(X, theta, y, alpha)
    return [theta, error_record]
    

In [9]:
'''
Verify gradients computed with normal equation

y = X * Theta
X' * y = X' * X * Theta
X' * y = (X' * X) * Theta
Theta = ((X' * X) ** -1) * X' * y

'''
def verifyTheta(X, y):
    thetaComputed = np.dot(np.linalg.pinv(np.dot(X.T, X)), np.dot(X.T, y))
    return thetaComputed

In [11]:
'''
Initialize weights and compare results of gradient descent and normal equation
'''

for i in range(weights.shape[0]):
    weights[i] = random.random()

XTrain = normalizeFeatures(dfTrain)
XTest = normalizeFeatures(dfTest)

XTemp = np.ones((XTrain.shape[0], XTrain.shape[1]+1))
XTemp[:,1:] = XTrain
XTrain = XTemp

XTemp2 = np.ones((XTest.shape[0], XTest.shape[1]+1))
XTemp2[:,1:] = XTest
XTest = XTemp2

features = XTrain[:,0:2]
pred = XTrain[:,2]
thetaComputed = verifyTheta(features, pred)
print (thetaComputed)
[theta1, error_record1] = batch_gradient_descent(features, weights, pred, 1000, 0.01)
print (theta1)
[theta2, error_record2] = stochastic_gradient_descent(features, weights, pred, 1000, 0.01)
print (theta2)


[ -5.19323222e-18   3.51799097e-01]
[  8.36697347e-06   3.51809137e-01]
[  1.80362523e-16   3.51799097e-01]
